In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.utils.data as data
from torch.optim import SGD
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
import random
import torch.utils.data as torchdata
from torch.utils.data import SubsetRandomSampler
import matplotlib.pyplot as plt
import time
# from train_model import train_model
# from test_model import test_model
%matplotlib inline
import pickle
from data import ImageNetCDataset

In [2]:
import tent

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

weather = ['snow', 'fog', 'spatter']
imagebase = '/local/rcs/ll3504/datasets/'
dataset_name = 'imagenetc'
tr_dataset = ImageNetCDataset(imagebase + dataset_name, weather, 1, transform=data_transforms['train'])


DEBUGGING: lengh of default self.samples = 50000
DEBUGGING: lengh of second corruption samples = 0
DEBUGGING: the root of the imagefolder class is: /local/rcs/ll3504/datasets/imagenetc/snow/1


In [5]:
train_loader = torchdata.DataLoader(tr_dataset, batch_size=8, shuffle=False)


In [6]:
len(train_loader)

AssertionError: 

In [7]:
for image, label in train_loader:
    print("Doing nothing lol")
#     image, label

IndexError: list index out of range

In [5]:
def get_dataset(path='/database', dataset_name='caltech-256-common'):
    # No holdout testing data. train and test data are the same, but different transformation
    data_transforms = {
        'train': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
        'test': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    }

v    te_dataset = datasets.ImageFolder(path + dataset_name + '/', data_transforms['test'])

    return tr_dataset, te_dataset

In [52]:
def split_dataset(train_dataset, test_dataset, valid_size=0.02, batch_size=128, train_size = 128):
    '''
    This function splits dataset into train, val, and test sets, and return train, val, test dataloaders.
    Val and Test loaders are the same

    '''
    
    # what does the len function gives?
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
#     print("DEBUGGING: split =", split)
    random.shuffle(indices)
    train_idx, valid_idx = indices[split:split+train_size], indices[:split]
#     print("DEBUGGING: data size = ", len(train_idx), len(valid_idx))
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    
    # print("DEBUGGING: the train_ind are:", len(train_idx))


    train_loader = torchdata.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=48, pin_memory=True, drop_last=False, sampler = train_sampler)
    test_loader = torchdata.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=48, pin_memory=True, drop_last=False, sampler = valid_sampler)
    dataloaders = {'train': train_loader,
                   'val': test_loader,
                   'test': test_loader}
    dataset_sizes ={'train': train_size, #int(np.floor((1-valid_size) * num_train)),
                    'val': int(np.floor(valid_size * num_train)),
                    'test': int(np.floor(valid_size * num_train))}
    return dataloaders, dataset_sizes

In [53]:
imagebase = '/local/rcs/ll3504/datasets/'

In [54]:

corruption = ['fog', 'snow', 'spatter', 'gaussian_blur', 'gaussian_noise', 'brightness']
weather = ['fog', 'snow', 'spatter', 'frost']
digital = ['gaussian_blur', 'brightness', 'defocus_blur', 'contrast'] # NO gaussian noise in the files


In [55]:
def get_imagenetc(imagebase, batch_size=128, sample_size = 128, corruption=corruption):
    '''
    Returns:
        ref_dataloaders:          ImageNet original validation data, as a reference
        ref_dataset_sizes:        1000, not the sizes of the real dataset in the ref_loader, probs used downstream
        corrupted_dataloaders:    A list of corrupted dataloaders, each element in a list represetns the data loaders
                                  for one corruption type. Each element contains ['train']['val']['test'] loaders
        corrupted_dataset_sizes:  A list of dictionaries of the sizes of each loaders for each corruption
        corruption:               A list of corruption names, in the same order of the corrupted_dataloaders
    '''
    corrupted_dataloaders = {}
    
    imagenet_val = datasets.ImageNet(imagebase+'imagenetc/', split='val', transform=transforms.Compose([
            transforms.Resize([224, 224]),
            transforms.CenterCrop((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
                                   target_transform=None)#, download=False)
    
    random_indices = random.sample(range(0, len(imagenet_val)), int(len(imagenet_val)*0.02))
    imagenet_val_subset = data.Subset(imagenet_val, random_indices)
    clean_val_loader = torch.utils.data.DataLoader(imagenet_val_subset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=48)
#     ref_dataloaders = { 'val': val_loader,
#                        'test': val_loader}
#     ref_dataset_sizes ={'val': int(len(val_loader.dataset)),
#                         'test': int(len(val_loader.dataset))}
    
    # for every type of corruption, go to the specified severity folder
    for corr in corruption:
        dataloader_all_sev = []

        for severity in range(1,6):
            
            dataset_name = 'imagenetc/' + corr + '/' + str(severity)
            # Get dataset from folder
            corr_trian_images, corr_test_images = get_dataset(imagebase, dataset_name)

            # Get corruption-specific train, val, test loader
                # train: training data, non-overlap with val/test
                # val: non-overlap with train, same as test
                # test: non-overlap with train, same as test

            corr_dataloaders, _ = split_dataset(corr_trian_images, corr_test_images, valid_size=0.02, batch_size=batch_size, train_size=sample_size)

            dataloader_all_sev.append(corr_dataloaders)
            corrupted_dataloaders[corr] = dataloader_all_sev
        
        
#     return ref_dataloaders, ref_dataset_sizes, corrupted_dataloaders, corrupted_dataset_sizes, corruption
    return clean_val_loader, corrupted_dataloaders




In [56]:
clean_loader, imc_loaders = get_imagenetc(imagebase, batch_size=64, sample_size = 49000, corruption=(weather+digital))
imc_loaders

DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 1000
DEBUGGING: split = 1000
DEBUGGING: data size =  49000 10

{'fog': [{'train': <torch.utils.data.dataloader.DataLoader at 0x7f02914603a0>,
   'val': <torch.utils.data.dataloader.DataLoader at 0x7f02914607c0>,
   'test': <torch.utils.data.dataloader.DataLoader at 0x7f02914607c0>},
  {'train': <torch.utils.data.dataloader.DataLoader at 0x7f0291464be0>,
   'val': <torch.utils.data.dataloader.DataLoader at 0x7f02914649d0>,
   'test': <torch.utils.data.dataloader.DataLoader at 0x7f02914649d0>},
  {'train': <torch.utils.data.dataloader.DataLoader at 0x7f0291464d90>,
   'val': <torch.utils.data.dataloader.DataLoader at 0x7f0291464f40>,
   'test': <torch.utils.data.dataloader.DataLoader at 0x7f0291464f40>},
  {'train': <torch.utils.data.dataloader.DataLoader at 0x7f0291464070>,
   'val': <torch.utils.data.dataloader.DataLoader at 0x7f0291464e80>,
   'test': <torch.utils.data.dataloader.DataLoader at 0x7f0291464e80>},
  {'train': <torch.utils.data.dataloader.DataLoader at 0x7f02914dd490>,
   'val': <torch.utils.data.dataloader.DataLoader at 0x7f02914dd1

In [57]:
clean_loader

In [58]:
def get_imagenet(imagebase, batch_size=128, sample_size = 128):
    # this is the imageNet validation data
    imagenet_val = datasets.ImageNet(imagebase+'imagenetc/', split='val', transform=transforms.Compose([
#             transforms.Resize([224, 224]),
#             transforms.CenterCrop((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]),
                                   target_transform=None)#, download=False)
    
    # TODO: subsample some size of ImageNet training data as source
        # Doesn't need this step
#     print("DEBUGGING: imagenet_val size is:", len(imagenet_val))
    
    random_indices = random.sample(range(0, len(imagenet_val)), int(len(imagenet_val)*0.02))
#     print("DEBUGGING: random indices are:", len(random_indices))
    imagenet_val_subset = data.Subset(imagenet_val, random_indices)
    val_loader = torch.utils.data.DataLoader(imagenet_val_subset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=48)
    ref_dataloaders = { 'val': val_loader,
                       'test': val_loader}
    ref_dataset_sizes ={'val': int(len(val_loader.dataset)),
                        'test': int(len(val_loader.dataset))}
    
    return ref_dataloaders

In [59]:
def online_evaluate(corrupt_loaders, corrutpion, severity, lr):
    start = time.time()
    resnet50 = models.resnet50(pretrained=True)
    resnet50 = tent.configure_model(resnet50)
    params, param_names = tent.collect_params(resnet50)
    optimizer = SGD(params, lr=lr)
    tented_resnet50 = tent.Tent(resnet50, optimizer).to(device)

    num_correct, num_samples = 0., 0.
    
    trainloader = corrupt_loaders[corrutpion][severity-1]['train']

    for images, targets in trainloader:
        logits = tented_resnet50(images.to(device))
        predictions = logits.argmax(dim=1)
        num_correct += (predictions.detach().cpu() == targets).float().sum()
        num_samples += len(targets)

    adapt_time = time.time() - start
    accuracy = num_correct / num_samples
    print(f"Acc =: {num_correct:#5.0f}/{num_samples:#5.0f} ({100 * accuracy: .2f} %), \
            Error =: {100 * (1 - accuracy): .2f} %")
    print(f"Adaptation time for one epoch on {num_samples} images takes {adapt_time}s")
    return tented_resnet50, accuracy, adapt_time

In [60]:
def offline_validate(model, corrupt_loaders, corruption, severity, baseline=False):
    if baseline:
        model.eval()
    else:
        model.offline_validation()
    valloader = corrupt_loaders[corruption][severity-1]['val']
    
    num_correct, num_samples = 0., 0.

#     with model.no_grad():
    for images, targets in valloader:
        logits = model(images.to(device))
        predictions = logits.argmax(dim=1)
        num_correct += (predictions.detach().cpu() == targets).float().sum()
        num_samples += len(targets)

    accuracy = num_correct / num_samples
    print(f"Validation Acc =: {num_correct:#5.0f}/{num_samples:#5.0f} ({100 * accuracy: .2f} %), \
            Validation Error =: {100 * (1 - accuracy): .2f} %")
    
    return accuracy

# Val Acc > Train?

## Is the model still updating in validation phase?

In [25]:
# turn model on offline validation mode, check if the corrupted set is adapted
lr = 0.00025
bs = 64

# ONLINE adaptation adaptation on the "training set"
resnet50 = models.resnet50(pretrained=True)
resnet50 = tent.configure_model(resnet50)
params, param_names = tent.collect_params(resnet50)
optimizer = SGD(params, lr=lr)
tented_resnet50 = tent.Tent(resnet50, optimizer).to(device)

tented_resnet50.offline_validation()
valloader = imc_loaders['fog'][2]['train']

num_correct, num_samples = 0., 0.

# tented_resnet50.model.eval()
#     with model.no_grad():
for images, targets in valloader:
    logits = tented_resnet50(images.to(device))
    predictions = logits.argmax(dim=1)
    num_correct += (predictions.detach().cpu() == targets).float().sum()
    num_samples += len(targets)

accuracy = num_correct / num_samples
print(f"Validation Acc =: {num_correct:#5.0f}/{num_samples:#5.0f} ({100 * accuracy: .2f} %), \
        Validation Error =: {100 * (1 - accuracy): .2f} %")

Traceback (most recent call last):
  File "/home/zw2774/bin/anaconda3/lib/python3.9/multiprocessing/queues.py", line 241, in _feed
    close()
  File "/home/zw2774/bin/anaconda3/lib/python3.9/multiprocessing/connection.py", line 182, in close
    self._close()
  File "/home/zw2774/bin/anaconda3/lib/python3.9/multiprocessing/connection.py", line 366, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor
Traceback (most recent call last):
  File "/home/zw2774/bin/anaconda3/lib/python3.9/multiprocessing/queues.py", line 241, in _feed
    close()
  File "/home/zw2774/bin/anaconda3/lib/python3.9/multiprocessing/connection.py", line 182, in close
    self._close()
  File "/home/zw2774/bin/anaconda3/lib/python3.9/multiprocessing/connection.py", line 366, in _close
    _close(self._handle)
Traceback (most recent call last):
  File "/home/zw2774/bin/anaconda3/lib/python3.9/multiprocessing/queues.py", line 241, in _feed
    close()
  File "/home/zw2774/bin/anaconda3/lib/pytho

Validation Acc =: 22852./48960. ( 46.67 %),         Validation Error =:  53.33 %


In [61]:
valloader = imc_loaders['fog'][2]['train']


In [62]:
# evaluate on the corrupted set on baseline resnet50 model, the result should be comparable same as above cell
num_correct, num_samples = 0., 0.

baseline = models.resnet50(pretrained=True).to(device)
baseline.eval()
for images, targets in valloader:
    logits = baseline(images.to(device))
    predictions = logits.argmax(dim=1)
    num_correct += (predictions.detach().cpu() == targets).float().sum()
    num_samples += len(targets)

accuracy = num_correct / num_samples
print(f"Validation Acc =: {num_correct:#5.0f}/{num_samples:#5.0f} ({100 * accuracy: .2f} %), \
        Validation Error =: {100 * (1 - accuracy): .2f} %")

Validation Acc =: 22841./49000. ( 46.61 %),         Validation Error =:  53.39 %


In [31]:
# evaluate on the clean on baseline resnet50 model, the result should be a lot better than above cell
num_correct, num_samples = 0., 0.

baseline = models.resnet50(pretrained=True).to(device)
baseline.eval()
for images, targets in clean_loader:
    logits = baseline(images.to(device))
    predictions = logits.argmax(dim=1)
    num_correct += (predictions.detach().cpu() == targets).float().sum()
    num_samples += len(targets)

accuracy = num_correct / num_samples
print(f"Validation Acc =: {num_correct:#5.0f}/{num_samples:#5.0f} ({100 * accuracy: .2f} %), \
        Validation Error =: {100 * (1 - accuracy): .2f} %")

Validation Acc =:  759./1000. ( 75.90 %),         Validation Error =:  24.10 %


## Check if my offline_validate function is correctly implemented

In [24]:
def offline_validate_train(model, corrupt_loaders, corruption, severity, baseline=False):
    if baseline:
        model.eval()
    else:
        model.offline_validation()
    valloader = corrupt_loaders[corruption][severity-1]['train']
    
    num_correct, num_samples = 0., 0.

#     with model.no_grad():
    for images, targets in valloader:
        logits = model(images.to(device))
        predictions = logits.argmax(dim=1)
        num_correct += (predictions.detach().cpu() == targets).float().sum()
        num_samples += len(targets)

    accuracy = num_correct / num_samples
    print(f"Validation Acc on Train =: {num_correct:#5.0f}/{num_samples:#5.0f} ({100 * accuracy: .2f} %), \
            Validation Error =: {100 * (1 - accuracy): .2f} %")
    
    return accuracy

In [26]:
# Debug the offline_validate function, running on the training & validation data
# expect similar performance on both set
lr = 0.00025
bs = 64

resnet50 = models.resnet50(pretrained=True)
resnet50 = tent.configure_model(resnet50)
params, param_names = tent.collect_params(resnet50)
optimizer = SGD(params, lr=lr)
tented_resnet50 = tent.Tent(resnet50, optimizer).to(device)

for corr in weather:
    online_acc_sum = 0
    validate_acc_sum = 0

    for severity in range(1,6):
        print(f"Corruption: {corr}, Severity: {severity}")
        
        # online adaptation:
#         start = time.time()
        accuracy = offline_validate_train(tented_resnet50, imc_loaders, corr, severity)
#         adapt_time = time.time() - start
        online_acc_sum += accuracy
        
        # offline validation:
        val_acc = offline_validate(tented_resnet50, imc_loaders, corr, severity)
        validate_acc_sum += val_acc
        
    print(f"Averag online accuracy for {corr}: {100 * (online_acc_sum / 5): .2f} %, \
            Averag online error for {corr}: {100 * (1 - online_acc_sum / 5): .2f} % ")
    print()
    print(f"Averag validation accuracy for {corr}: {100 * (validate_acc_sum / 5): .2f} %, \
            Averag validation error for {corr}: {100 * (1 - validate_acc_sum / 5): .2f} % ")
    print("======================================================================================")



Corruption: fog, Severity: 1
Validation Acc on Train =: 30251./48960. ( 61.79 %),             Validation Error =:  38.21 %
Validation Acc =:  612./ 960. ( 63.75 %),             Validation Error =:  36.25 %
Corruption: fog, Severity: 2
Validation Acc on Train =: 27364./48960. ( 55.89 %),             Validation Error =:  44.11 %
Validation Acc =:  539./ 960. ( 56.15 %),             Validation Error =:  43.85 %
Corruption: fog, Severity: 3
Validation Acc on Train =: 22855./48960. ( 46.68 %),             Validation Error =:  53.32 %
Validation Acc =:  426./ 960. ( 44.38 %),             Validation Error =:  55.62 %
Corruption: fog, Severity: 4
Validation Acc on Train =: 19800./48960. ( 40.44 %),             Validation Error =:  59.56 %
Validation Acc =:  371./ 960. ( 38.65 %),             Validation Error =:  61.35 %
Corruption: fog, Severity: 5
Validation Acc on Train =: 11955./48960. ( 24.42 %),             Validation Error =:  75.58 %
Validation Acc =:  241./ 960. ( 25.10 %),            

# Replicate TENT

In [ ]:
lr = 0.00025
bs = 64

# ONLINE adaptation adaptation on the "training set"

for corr in weather:
    online_acc_sum = 0
    validate_acc_sum = 0

    for severity in range(1,6):
        print(f"Corruption: {corr}, Severity: {severity}")
        
        # online adaptation:
        start = time.time()
        adapted_model, accuracy, _ = online_evaluate(imc_loaders, corr, severity, lr)
        adapt_time = time.time() - start
        online_acc_sum += accuracy
        
        # offline validation:
        val_acc = offline_validate(adapted_model, imc_loaders, corr, severity)
        validate_acc_sum += val_acc
        
    print(f"Averag online accuracy for {corr}: {100 * (online_acc_sum / 5): .2f} %, \
            Averag online error for {corr}: {100 * (1 - online_acc_sum / 5): .2f} % ")
    print()
    print(f"Averag validation accuracy for {corr}: {100 * (validate_acc_sum / 5): .2f} %, \
            Averag validation error for {corr}: {100 * (1 - validate_acc_sum / 5): .2f} % ")
    print("======================================================================================")



Corruption: fog, Severity: 1
Acc =: 33750./48960. ( 68.93 %),             Error =:  31.07 %
Adaptation time for one epoch on 48960.0 images takes 136.8747386932373s
Validation Acc =:  656./ 960. ( 68.33 %),             Validation Error =:  31.67 %
Corruption: fog, Severity: 2
Acc =: 32625./48960. ( 66.64 %),             Error =:  33.36 %
Adaptation time for one epoch on 48960.0 images takes 134.08197832107544s
Validation Acc =:  667./ 960. ( 69.48 %),             Validation Error =:  30.52 %
Corruption: fog, Severity: 3
Acc =: 30818./48960. ( 62.95 %),             Error =:  37.05 %
Adaptation time for one epoch on 48960.0 images takes 132.2472791671753s
Validation Acc =:  594./ 960. ( 61.87 %),             Validation Error =:  38.13 %
Corruption: fog, Severity: 4
Acc =: 29473./48960. ( 60.20 %),             Error =:  39.80 %
Adaptation time for one epoch on 48960.0 images takes 133.57329559326172s
Validation Acc =:  605./ 960. ( 63.02 %),             Validation Error =:  36.98 %
Corrup

In [37]:
weather_us = ['fog', 'snow', 'spatter']
digital_us = ['gaussian_blur', 'brightness']
weather_us + digital_us

['fog', 'snow', 'spatter', 'gaussian_blur', 'brightness']

In [23]:
cross_valid_acc = {}

for corr in weather_us + digital_us:
    online_acc_sum = 0
    cross_valid_acc[corr] = {}
    
    for severity in [3]:
        print(f"Corruption: {corr}, Severity: {severity}")

        # online adaptation:
        adapted_model, accuracy, _ = online_evaluate(imc_loaders, corr, severity, lr)
        online_acc_sum += accuracy
        
        
        # offline validation:
        for val_corr in weather_us + digital_us:
            print(f"cross-validating on corruption = {val_corr}")
            val_acc = offline_validate(adapted_model, imc_loaders, val_corr, severity)
            cross_valid_acc[corr][val_corr] = val_acc

#     print(f"Averag online accuracy for {corr}: {100 * (online_acc_sum / 5): .2f} %, \
#             Averag online error for {corr}: {100 * (1 - online_acc_sum / 5): .2f} % ")
#     print()
#     print(f"Averag validation accuracy for {corr}: {100 * (validate_acc_sum / 5): .2f} %, \
#             Averag validation error for {corr}: {100 * (1 - validate_acc_sum / 5): .2f} % ")
    print("======================================================================================")

Corruption: fog, Severity: 3
Acc =: 30820./48960. ( 62.95 %),             Error =:  37.05 %
Adaptation time for one epoch on 48960.0 images takes 132.4763262271881s
cross-validating on corruption = fog
Validation Acc =:  607./ 960. ( 63.23 %),             Validation Error =:  36.77 %
cross-validating on corruption = snow
Validation Acc =:  439./ 960. ( 45.73 %),             Validation Error =:  54.27 %
cross-validating on corruption = spatter
Validation Acc =:  566./ 960. ( 58.96 %),             Validation Error =:  41.04 %
cross-validating on corruption = gaussian_blur
Validation Acc =:  436./ 960. ( 45.42 %),             Validation Error =:  54.58 %
cross-validating on corruption = brightness
Validation Acc =:  675./ 960. ( 70.31 %),             Validation Error =:  29.69 %
Corruption: snow, Severity: 3
Acc =: 24388./48960. ( 49.81 %),             Error =:  50.19 %
Adaptation time for one epoch on 48960.0 images takes 133.70379257202148s
cross-validating on corruption = fog
Validatio

In [38]:
severity = 3
baseline_valid_acc = {}

vanilla = models.resnet50(pretrained=True).to(device)
for val_corr in weather_us + digital_us:
    print(f"cross-validating on corruption = {val_corr}")
    val_acc = offline_validate(vanilla, imc_loaders, val_corr, severity, baseline=True)
    baseline_valid_acc[val_corr] = val_acc

cross-validating on corruption = fog
Validation Acc =:  446./ 960. ( 46.46 %),             Validation Error =:  53.54 %
cross-validating on corruption = snow
Validation Acc =:  337./ 960. ( 35.10 %),             Validation Error =:  64.90 %
cross-validating on corruption = spatter
Validation Acc =:  485./ 960. ( 50.52 %),             Validation Error =:  49.48 %
cross-validating on corruption = gaussian_blur
Validation Acc =:  392./ 960. ( 40.83 %),             Validation Error =:  59.17 %
cross-validating on corruption = brightness
Validation Acc =:  690./ 960. ( 71.88 %),             Validation Error =:  28.12 %


In [39]:
49000 // 64

765

In [40]:
765 * 64

48960